<a href="https://colab.research.google.com/github/spdzh/Ship-Movement-Scheduling/blob/main/GA_Restriction2%263.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Packages

In [ ]:
pip install pulp

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from numpy.random import default_rng
import time
import heapq
from pulp import GLPK
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

Import Data

In [3]:
dataset = pd.read_csv('Real_20.csv')

change date to number

In [4]:
def TimetoNum(x):
    utc_time = datetime.strptime(x, '%m/%d/%Y %H:%M')
    x= datetime.strftime(utc_time, '%Y-%m-%d %H:%M')
    y=datetime.fromisoformat(x).timestamp()
    return y

In [5]:
dataset["PilotOrderTime"]=dataset["PilotOrderTime"].apply(TimetoNum)

Fitness Function

In [6]:
def Fitness(dataset, range1):
    model = LpProblem(name="small-problem", sense=LpMinimize)
        #Scheduled= dataset["PilotOrderTime"].copy()
    Scheduled = {i: LpVariable(name=('Scheduled %i' % i), lowBound=dataset["PilotOrderTime"][i]) for i in range(len(dataset))}
    x = {j: LpVariable(name=f"x{j}", lowBound=0, upBound = 1, cat="Integer") for j in range(100)}
    y = {o: LpVariable(name=f"y{o}", lowBound=0, upBound = 1, cat="Integer") for o in range(100)}

    l=0            
    k=0
    kk=0
    m=10000000
    for i in range1:
        model += Scheduled[i]>= dataset["PilotOrderTime"][i]
        model += Scheduled[i]<= 1533083200+m*y[kk]
        model += Scheduled[i]>= 1533084400+m*(y[kk]-1)
        kk=kk+1
        l=l+1
        if (dataset["d4"][i]>0):
            if ((dataset["Type"][i]== 2 and dataset["Beam"][i]>=120)or (dataset["Beam"][i]>=150)):
                model += Scheduled[i]>= 1533104500
                model += Scheduled[i]<= 1533146400
      
        for j in range1[l:]:
            if (dataset["Direction"][i]==1 and (dataset["Direction"][j]!= dataset["Direction"][i])):
                if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]> dataset["d2"][i])):
                    LB1= Scheduled[i]+ (dataset["d1"][i]/dataset["Speed"][i])*60*60
                    LB2= (dataset["d2"][j]/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d2"][i]/dataset["Speed"][i])*60*60
                    UB2= ((dataset["d2"][j]-dataset["d2"][i])/dataset["Speed"][j])*60*60
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1
                                            
                    
                if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]<= dataset["d2"][i])):
                    LB1= Scheduled[i]+(dataset["d1"][i]/dataset["Speed"][i])*60*60
                    LB2= (dataset["d2"][j]/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d2"][j]/dataset["Speed"][i])*60*60
                    UB2= 0
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1
                    
                if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]> dataset["d4"][i])):
                    LB1= Scheduled[i]+ (dataset["d3"][i]/dataset["Speed"][i])*60*60
                    LB2= (dataset["d4"][j]/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d4"][i]/dataset["Speed"][i])*60*60
                    UB2= ((dataset["d4"][j]-dataset["d4"][i])/dataset["Speed"][j])*60*60
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1
                                            
                    
                if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]<= dataset["d4"][i])):
                    LB1= Scheduled[i]+(dataset["d3"][i]/dataset["Speed"][i])*60*60
                    LB2= (dataset["d4"][j]/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d4"][j]/dataset["Speed"][i])*60*60
                    UB2= 0
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1     
                            
            if (dataset["Direction"][i]==0 and (dataset["Direction"][j]!= dataset["Direction"][i])):
                if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]> dataset["d2"][i])):
                    LB1= Scheduled[i]
                    LB2= ((dataset["d1"][j]+dataset["d2"][i])/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d2"][i]/dataset["Speed"][i])*60*60
                    UB2= (dataset["d1"][j]/dataset["Speed"][j])*60*60
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1
                            
                    
                if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]<= dataset["d2"][i])):
                    LB1= Scheduled[i]+((dataset["d2"][i]-dataset["d2"][j])/dataset["Speed"][i])*60*60
                    LB2= ((dataset["d1"][j]+dataset["d2"][j])/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d2"][j]/dataset["Speed"][i])*60*60
                    UB2= (dataset["d1"][j]/dataset["Speed"][j])*60*60
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1

                if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]> dataset["d4"][i])):
                    LB1= Scheduled[i]
                    LB2= ((dataset["d3"][j]+dataset["d4"][i])/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d4"][i]/dataset["Speed"][i])*60*60
                    UB2= (dataset["d3"][j]/dataset["Speed"][j])*60*60
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1
                            
                    
                if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]<= dataset["d4"][i])):
                    LB1= Scheduled[i]+((dataset["d4"][i]-dataset["d4"][j])/dataset["Speed"][i])*60*60
                    LB2= ((dataset["d3"][j]+dataset["d4"][j])/dataset["Speed"][j])*60*60
                    UB1= LB1+ (dataset["d4"][j]/dataset["Speed"][i])*60*60
                    UB2= (dataset["d3"][j]/dataset["Speed"][j])*60*60
                    model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                    model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                    k=k+1
    
    xy= lpSum(Scheduled)
    yx= lpSum(dataset["PilotOrderTime"])
    model += xy-yx
    status = model.solve()
    fitness= model.objective.value()
    scheme= model.variables()

    return (fitness, scheme)


In [7]:
import datetime
datetime.datetime.fromtimestamp(1533146400)

datetime.datetime(2018, 8, 1, 18, 0)

In [ ]:
print(model)
print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

Generating population

In [9]:
rng = default_rng()
population=[]

for i in range(12):
    numbers = np.array(rng.choice(len(dataset), size=len(dataset), replace=False))
    population.append(numbers)

Mutation

In [10]:
def mutation(dataset, x):
    rand1= np.random.randint(0, len(dataset))
    rand2= np.random.randint(rand1, len(dataset))
    y= x.copy()
    y[rand1]= x[rand2].copy()
    y[rand2]= x[rand1].copy()
    return y

Cross-Over

In [11]:
def crossover(dataset, population):
    offspring= []
    for i in range(4):
        rand1= np.random.randint(0, len(dataset))
        rand2= np.random.randint(rand1, len(dataset))
        x=population[i].copy()
        x[0:(len(dataset)-rand2)]= population[i][rand2:].copy()
        x[(len(dataset)-rand2):(len(dataset)-rand2)+(rand2-rand1)]= population[i][rand1:rand2].copy()
        x[(len(dataset)-rand2)+(rand2-rand1):]= population[i][0:rand1].copy()
        rad= np.random.rand()
        if (rad<0.1):
            x=mutation(dataset, x)
        offspring.append(x)
        
        rand1= np.random.randint(0, len(dataset))
        rand2= np.random.randint(rand1, len(dataset))
        x=population[i].copy()
        x[0:(len(dataset)-rand2)]= population[i][rand2:].copy()
        x[(len(dataset)-rand2):(len(dataset)-rand2)+(rand2-rand1)]= population[i][rand1:rand2].copy()
        x[(len(dataset)-rand2)+(rand2-rand1):]= population[i][0:rand1].copy()
        rad= np.random.rand()
        if (rad<0.1):
            x=mutation(dataset, x)
        offspring.append(x)
        
    return offspring

**Genetic Algorithm**

In [12]:
start_time= time.clock()
sol_glo= 10000000000000
for r in range(10):
    rng = default_rng()
    population=[]
    for i in range(10):
        numbers = np.array(rng.choice(len(dataset), size=len(dataset), replace=False))
        population.append(numbers)

    for j in range(10):
        cost= []
        for i in range(len(population)):
            Scheduled= dataset["PilotOrderTime"].copy()
            cost.append(Fitness(dataset,population[i])[0])

        sol_loc= min(cost)
        if sol_loc<= sol_glo:
            sol_glo= sol_loc
            sol_index= cost.index(min(cost))
            sol= population[sol_index]
            final_scheme= Fitness(dataset,population[sol_index])[1]

        a= heapq.nsmallest(4, ((k, yy) for yy, k in enumerate(cost)))
        Z=[]
        for i in range(len(a)):
            Z.append(population[a[i][1]])

        Q= crossover(dataset, Z)   
        for i in range(2):
            Q.append(Z[i])

duration= time.clock()- start_time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [13]:
print(duration)

32.805688


In [14]:
print(Scheduled[1])

1533081660.0


In [15]:
import datetime
timestamp = .apply(datetime.datetime.fromtimestamp)
print(timestamp)

SyntaxError: ignored

In [16]:
print(dataset["PilotOrderTime"][15])

1533103200.0


In [17]:
print(sol_glo)
print(sol)
print(dataset["PilotOrderTime"][7])
for var in final_scheme:
    print(f"{var.name}: {var.value()}")

2820.0
[17  5 15  2  9 12  1 10 18 16 11  0 13  3 14 19  4  6  7  8]
1533090600.0
Scheduled_0: 1533081700.0
Scheduled_1: 1533081700.0
Scheduled_10: 1533097800.0
Scheduled_11: 1533099600.0
Scheduled_12: 1533099600.0
Scheduled_13: 1533099600.0
Scheduled_14: 1533103200.0
Scheduled_15: 1533103700.0
Scheduled_16: 1533105200.0
Scheduled_17: 1533105000.0
Scheduled_18: 1533105000.0
Scheduled_19: 1533108600.0
Scheduled_2: 1533081700.0
Scheduled_3: 1533084400.0
Scheduled_4: 1533084400.0
Scheduled_5: 1533085200.0
Scheduled_6: 1533088800.0
Scheduled_7: 1533090600.0
Scheduled_8: 1533090600.0
Scheduled_9: 1533097800.0
x0: 0.0
x1: 0.0
x10: 1.0
x11: 0.0
x12: 1.0
x13: 0.0
x14: 0.0
x15: 0.0
x16: 0.0
x17: 0.0
x18: 0.0
x19: 0.0
x2: 0.0
x20: 0.0
x21: 0.0
x22: 0.0
x23: 1.0
x24: 1.0
x25: 1.0
x26: 1.0
x27: 1.0
x28: 0.0
x29: 0.0
x3: 0.0
x30: 0.0
x31: 0.0
x4: 0.0
x5: 1.0
x6: 1.0
x7: 0.0
x8: 1.0
x9: 1.0
y0: 1.0
y1: 1.0
y10: 1.0
y11: 0.0
y12: 1.0
y13: 1.0
y14: 1.0
y15: 1.0
y16: 1.0
y17: 1.0
y18: 1.0
y19: 1.0
y2: 

In [ ]:
import datetime
datetime.datetime.fromtimestamp(1533086200.0)

datetime.datetime(2018, 8, 1, 1, 16, 40)

In [18]:
start_time= time.clock()
from itertools import permutations 
l = list(permutations(range(0, 10))) 
print(l[1])
print(population[1])
duration1= time.clock()- start_time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


(0, 1, 2, 3, 4, 5, 6, 7, 9, 8)
[11  2 18  1 17 10  4  8 16 14 19  9 13  5  6 15  3  0  7 12]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [19]:
cost= 100000000
axe=0
for i in range(len(l)):
    Scheduled= dataset["PilotOrderTime"].copy()
    axe=Fitness(dataset,l[i])[0]
    if axe< cost:
        cost=axe
        sol_index= cost.index(min(cost))
            sol= population[sol_index]
            final_scheme= Fitness(dataset,population[sol_index])[1]


duration2= time.clock()- start_time

IndentationError: ignored

In [20]:
print(cost)
print(time.clock()-start_time)

[2820.0, 2820.0, 2820.0, 2820.0, 2820.0, 2820.0, 2820.0, 2820.0, 2820.0, 2820.0]
1.1049219999999949


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
